# Lab 2 - Naiwna klasyfikacja bayesowska i przetwarzanie języka naturalnego (NLP)

## Prawdopodobieństwo warunkowe

Prawdopodobieństwo warunkowe to wartość określająca szansę wystąpienia zdarzenia losowego A pod warunkiem
wystąpienia zdarzenia losowego B:

$$ P(A | B) = \frac{P(A \cap B)}{P(B)} $$

gdzie:
- $$ P(B) $$ oznacza prawdopodobieństwo wystąpienie zdarzenia losowego B
- $$ A \cap B $$ oznacza część wspólną zdarzeń losowych A i B
- $$ P(A \cap B) = \frac{|A \cap B|}{|A \cup B|} $$

co można przedstawić następująco w języku Python:

In [1]:
import email
import io
import re
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

a = {4, 6, 7}
b = {3, 6, 9}

a_and_b = a & b
a_or_b = a | b

p_a_and_b = len(a_and_b) / len(a_or_b)
p_b = len(b) / len(a_or_b)
p_a_if_b = p_a_and_b / p_b

print(f'Przestrzen zdarzen A: {a}')
print(f'Przestrzen zdarzen B: {b}')
print(f'Czesc wspolna zdarzen A i B: {a_and_b}')
print(f'Suma zdarzen A i B: {a_or_b}')
print(f'Prawdopodobienstwo czesci wspolnej zdarzen A i B: {p_a_and_b}')
print(f'Prawdopodobienstwo zdarzenia A pod warunkiem zajscia zdarzenia B: {p_a_if_b}')


Przestrzen zdarzen A: {4, 6, 7}
Przestrzen zdarzen B: {9, 3, 6}
Czesc wspolna zdarzen A i B: {6}
Suma zdarzen A i B: {3, 4, 6, 7, 9}
Prawdopodobienstwo czesci wspolnej zdarzen A i B: 0.2
Prawdopodobienstwo zdarzenia A pod warunkiem zajscia zdarzenia B: 0.33333333333333337


## Naiwny klasyfikator bayesowski

Klasyfikator probabilistyczny, oparty na założeniu o wzajemnej niezależności atrybutów systemu decyzyjnego.

Wynik predykcji modelu jest prawdopodobieństwem warunkowym przynależności obiektu do klasy decyzyjnej $$ C $$ przy użyciu
atrybutów $$ A_1, A_2, ..., A_n $$, co można zdefiniować następująco:

$$ p(C | A_1, A_2, ... A_n) = \frac{p(C) p(A_1, A_2, ..., A_n | C)}{p(A_1, A_2, ..., A_n)} $$

## Przykład przydzielenia decyzji metodą Naiwnej Klasyfikacji Bayesowskiej

Rozważymy przykład Naiwnej Klasyfikacji Bayesowskiej na podstawie następującego systemu decyzyjnego zawierającego słowa,
 liczby ich wystąpień w danej klasie dokumentów tekstowych oraz etykietę klasy dokumentu:

| Słowo     | Liczba wystąpień | Klasa dokumentu          |
|-----------|------------------|--------------------------|
| witaj     | 10               | Wiadomość treściwa (ham) |
| promocja  | 35               | Spam                     |
| cześć     | 27               | Wiadomość treściwa (ham) |
| spotkanie | 6                | Wiadomość treściwa (ham) |
| oferta    | 42               | Spam                     |
| cześć     | 12               | Spam                     |
| witaj     | 21               | Spam                     |
| jira      | 35               | Wiadomość treściwa (ham) |
| projekt   | 42               | Wiadomość treściwa (ham) |
| zakupy    | 10               | Spam                     |

Rozważmy przykładowy tekst "witaj, jak twoje zaangażowanie w projekt?"

Każdemu słowu dopasujemy warunkowe prawdopodobieństwo wystąpienia dla każdej klasy decyzyjnej.
Pomijamy słowa krótsze niż 4 znaki.

| Słowo   | Prawdopodobieństwo wystąpienia w klasie           |
|---------|---------------------------------------------------|
| witaj   | P(witaj\|ham) = 10/120; P(witaj\|spam) = 21/120   |
| projekt | P(projekt\|ham) = 42/120; P(projekt\|spam) = 0    |

Mając ustalone prawdopodobieństwa wystąpienia słów w poszczególnych klasach decyzyjnych możemy przejść
do obliczenia estymaty bayesowskiej dla całego zdania wobec każdej klasy decyzyjnej, która pozwoli przydzielić decyzję.

W celu wyznaczenia estymaty bayesowskiej dla danej klasy decyzyjnej obliczamy iloczyn prawdopodobieństw wystąpienia
każdego słowa. Jeżeli dane słowo nie występuje, przydzielamy bardzo małe prawdopodobieństwo wystąpienia, np 0.00001.

- $$E_{ham} = \frac{10}{120} * 0.00001 * 0.00001 * 0.00001 * \frac{42}{120} = 2.9166667 * 10^{-17}$$
- $$E_{spam} = \frac{21}{120} * 0.00001 * 0.00001 * 0.00001 * 0.00001 = 1.75 * 10^{-21}$$

Estymata dla klasy wiadomości treściwych jest większa niż dla klasy spamu, zatem metodą
Naiwnej Klasyfikacji Bayesowskiej przydzielamy decyzję, że analizowany tekst jest wiadomością treściwą.

### Zadanie

Przydzielić decyzję metodą Naiwnej Klasyfikacji Bayesowskiej (na podstawie powyższego systemu decyzyjnego) następującym wiadomościom:
- "spotkanie się przedłużyło, jak wrócę to zrobię zakupy"
- "cześć john, w naszym sklepie czeka na ciebie specjalna oferta"
- "kiedy dotrzemy na lotnisko?"


## Tokenizacja

Jeden z najbardziej podstawowych etapów przetwarzania języka naturalnego, który polega na podziale
sekwencji słów (np. zdania) na mniejsze jednostki zwane tokenami (np. słowa lub znaki).

Przykładowa tokenizacja słów przy użyciu języka Python:

In [2]:
seq = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.'
tokens_ = re.findall('\w+', seq)

tokens_

['Lorem',
 'ipsum',
 'dolor',
 'sit',
 'amet',
 'consectetur',
 'adipiscing',
 'elit',
 'sed',
 'do',
 'eiusmod',
 'tempor',
 'incididunt',
 'ut',
 'labore',
 'et',
 'dolore',
 'magna',
 'aliqua']

## Budowa filtra spamu przy użyciu Naiwnego klasyfikatora bayesowskiego

Jest to jeden z najbardziej typowych przykładów zastosowania Naiwnego klasyfikatora bayesowskiego.
Do celów uczenia klasyfikatora zostanie wykorzystany zbiór maili CSDMC2010, który zawiera 4327 wiadomości, z następującym podziałem:
- 1378: spam
- 2949: nie spam

Zbiór wiadomości znajduje się w folderze data.

### Klasa EmailObject

Celem klasy jest parsowanie oryginalnych wiadomości e-mail (plik *.eml) w celu wyodrębnienia tematu oraz treści.
Te atrybuty są wystarczające w celu dokonania klasyfikacji.

In [3]:
class EmailObject:
    def __init__(self, file, category=None):
        self.mail = email.message_from_file(file)
        self.category = category

    def subject(self):
        return self.mail.get('Subject')

    def body(self):
        content_type = self.mail.get_content_type()
        body = self.mail.get_payload(decode=False)

        if content_type == 'text/html':
            return BeautifulSoup(body, 'html.parser').text
        elif content_type == 'text/plain':
            return body
        else:
            return ''


email_obj0 = EmailObject(io.open('./data/TRAINING/TRAIN_00002.eml', 'r', encoding='latin-1'))

email_obj0.subject()

'Re: How to manage multiple Internet connections?'

### Klasa Tokenizer

Celem klasy Tokenizer jest tokenizacja tekstu znajdującego się w wiadomości e-mail.

In [4]:
class Tokenizer:
    NULL = u'\u0000'

    @staticmethod
    def tokenize(txt):
        return re.findall('\w+', txt.lower())

    @staticmethod
    def ngram(txt, n=2):
        s = txt.split(' ')
        result = []
        for i in range(1, n + 1):
            result.append([Tokenizer.NULL] * (n - i) + s)
        return list(zip(*result))

    @staticmethod
    def unique_tokenizer(txt):
        tokens = Tokenizer.tokenize(txt)
        return set(tokens)

seq = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.'
tokens_ = Tokenizer.tokenize(seq)
three_grams = Tokenizer.ngram(seq, 3)

print(f'Tokens: {tokens_}\n3-grams: {three_grams}')


Tokens: ['lorem', 'ipsum', 'dolor', 'sit', 'amet', 'consectetur', 'adipiscing', 'elit', 'sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua']
3-grams: [('\x00', '\x00', 'Lorem'), ('\x00', 'Lorem', 'ipsum'), ('Lorem', 'ipsum', 'dolor'), ('ipsum', 'dolor', 'sit'), ('dolor', 'sit', 'amet,'), ('sit', 'amet,', 'consectetur'), ('amet,', 'consectetur', 'adipiscing'), ('consectetur', 'adipiscing', 'elit,'), ('adipiscing', 'elit,', 'sed'), ('elit,', 'sed', 'do'), ('sed', 'do', 'eiusmod'), ('do', 'eiusmod', 'tempor'), ('eiusmod', 'tempor', 'incididunt'), ('tempor', 'incididunt', 'ut'), ('incididunt', 'ut', 'labore'), ('ut', 'labore', 'et'), ('labore', 'et', 'dolore'), ('et', 'dolore', 'magna'), ('dolore', 'magna', 'aliqua.')]


### Klasa SpamTrainer

Cele klasy są następujące:
- uczenie modelu
- klasyfikacja (przydzielanie decyzji)

Uczenie modelu będzie polegało na przechowywaniu liczebności słów występującej w każdej klasie decyzyjnej (spam/nie spam).
Zatem każde słowo będzie oddzielnym atrybutem w systemie treningowym.

In [5]:
class SpamTrainer:
    def __init__(self, training_files):
        self.categories = set()

        for category, file in training_files:
            self.categories.add(category)

        self.totals = defaultdict(float)
        self.training = {c: defaultdict(float)
                         for c in self.categories}
        self.to_train = training_files

    def total_for(self, category):
        return self.totals[category]

    def train(self):
        for category, file in self.to_train:
            with open(file, 'r', encoding='latin-1') as f:
                mail = EmailObject(f)
            self.categories.add(category)

            for token in Tokenizer.unique_tokenizer(mail.body()):
                self.training[category][token] += 1
                self.totals['_all'] += 1
                self.totals[category] += 1

        self.to_train = {}

    def score(self, mail):
        self.train()
        cat_totals = self.totals

        aggregates = {c: cat_totals[c] / cat_totals['_all']
                      for c in self.categories}

        for token in Tokenizer.unique_tokenizer(mail.body()):
            for cat in self.categories:
                value = self.training[cat][token]
                r = (value + 1) / (cat_totals[cat] + 1)
                aggregates[cat] *= r

        return aggregates

    def normalized_score(self, mail):
        score = self.score(mail)
        scoresum = sum(score.values())

        normalized = {cat: (agg / scoresum)
                      for cat, agg in score.items()}

        return normalized

    def preference(self):
        return sorted(self.categories, key=lambda cat: self.total_for(cat))

    class Classification:
        def __init__(self, guess, score):
            self.guess = guess
            self.score = score

        def __eq__(self, other):
            return self.guess == other.guess and self.score == other.score

    def classify(self, mail):
        score = self.score(mail)

        max_score = 0.0
        preference = self.preference()
        max_key = preference[-1]

        for k, v in score.items():
            if v > max_score:
                max_key = k
                max_score = v
            elif v == max_score and preference.index(k) > preference.index(max_key):
                max_key = k
                max_score = v
        return self.Classification(max_key, max_score)

training_data_ = (
    ('ham', './data/TRAINING/TRAIN_00002.eml'),
    ('spam', './data/TRAINING/TRAIN_00000.eml'),
    ('ham', './data/TRAINING/TRAIN_00006.eml'),
    ('spam', './data/TRAINING/TRAIN_00003.eml')
)

spam_trainer = SpamTrainer(training_data_)


email_obj0_classification_score = spam_trainer.classify(email_obj0)
print(f'Wynik klasyfikacji maila niebedacego spamem: {email_obj0_classification_score.guess}')

print('System treningowy utworzony w klasie SpamTrainer')
spam_trainer.training


Wynik klasyfikacji maila niebedacego spamem: ham
System treningowy utworzony w klasie SpamTrainer


{'ham': defaultdict(float,
             {'r': 1.0,
              'fuller': 1.0,
              'never': 1.0,
              'buckminster': 1.0,
              'scheme': 1.0,
              'want': 1.0,
              'something': 1.0,
              'all': 1.0,
              'usa': 1.0,
              'am': 1.0,
              'achievable': 1.0,
              '44': 1.0,
              'lawrence': 1.0,
              'wrote': 1.0,
              'hettinga': 1.0,
              'the': 1.0,
              'know': 1.0,
              'reason': 1.0,
              'first': 1.0,
              'our': 1.0,
              'we': 1.0,
              'stoical': 1.0,
              'no': 1.0,
              'shoes': 1.0,
              'and': 1.0,
              'supplying': 1.0,
              'ask': 1.0,
              'to': 1.0,
              'think': 1.0,
              'farquhar': 1.0,
              'only': 1.0,
              'ibuc': 1.0,
              'problem': 1.0,
              'finished': 1.0,
              '021

### Walidacja modelu

W celu walidacji modelu zostaną zdefiniowane 3 funkcje pomocnicze.

Dodatkowo zostaną wykorzystane przygotowane podzbiory wiadomości e-mail wraz z etykietami (fold1, fold2).
Każdy z nich może być odpowiednikiem systemu treningowego i systemu testowego - są podzielone w stosunku 50%:50%.

W celu dokładniejszej walidacji modelu zostaną użyte dodatkowe miary wydajności klasyfikacji binarnej:
- FPR (False Positive Rate), odsetek obiektów, które zostały falszywie sklasyfikowane jako pozytywne: $$FPR = \frac{|False Positives|}{|total|}$$
- FNR (False Negative Rate), odsetek obiektów, które zostały falszywie sklasyfikowane jako negatywne: $$FNR = \frac{|False Negatives|}{|total|}$$

In [6]:
def label_to_training_data(fold_file):
    """
    Funkcja zwraca wytrenowany model na wyznaczonym podzbiorze wiadomosci
    """

    training_data = []

    with open(fold_file, 'r') as f:
        for line in f:
            target, filepath = line.rstrip().split(' ')
            training_data.append([target, filepath])

    return SpamTrainer(training_data)


def parse_emails(keyfile):
    """
    Funkcja zwraca wyznaczony podzbior wiadomosci w postaci obiektow klasy EmailObject
    """

    emails = []

    with open(keyfile, 'r') as f:
        for line in f:
            label, file = line.rstrip().split(' ')

            with open(file, 'r', encoding='latin-1') as labelfile:
                emails.append(EmailObject(labelfile, category=label))

    return emails


def validate(trainer, set_of_emails):
    """
    Funkcja dokonuje walidacji wytrenowanego modelu (trainer)
    na podstawie zbioru oznaczonych wiadomosci (set_of_emails)
    """

    correct = 0
    false_positives = 0.0
    false_negatives = 0.0
    confidence = 0.0

    for mail in set_of_emails:
        classification = trainer.classify(mail)
        confidence += classification.score

        if classification.guess == 'spam' and mail.category == 'ham':
            false_positives += 1
        elif classification.guess == 'ham' and mail.category == 'spam':
            false_negatives += 1
        else:
            correct += 1

    total = false_positives + false_negatives + correct

    false_positive_rate = false_positives / total
    false_negative_rate = false_negatives / total
    error = (false_positives + false_negatives) / total

    return false_positive_rate, false_negative_rate, error


trainer = label_to_training_data('data/fold2.label')
emails = parse_emails('data/fold1.label')
fpr, fnr, err = validate(trainer, emails)

print(f'FPR: {fpr}, FNR: {fnr}, error: {err}, accuracy: {1 - err}')

FPR: 0.0018484288354898336, FNR: 0.20841035120147874, error: 0.21025878003696857, accuracy: 0.7897412199630314


Zadania:

1. Sprawdzić powyższe parametry klasyfikacji dla następujących podziałów wiadomości e-mail:
    - 55% system treningowy i 45% system testowy
    - 60% system treningowy i 40% system testowy
    - 65% system treningowy i 35% system testowy
    - 70% system treningowy i 30% system testowy
    - 75% system treningowy i 25% system testowy
    - 80% system treningowy i 20% system testowy
Dodatkowo, przy każdym podziale pomieszać kolejność wiadomości, zachowując jednocześnie przypisanie etykiet.
W tym celu należy przygotować odpowiedni skrypt tworzący pliki analogiczne do data/fold1.label i data/fold2.label

2. Sprawdzić działanie modelu na zbiorze maili SpamAssasin: https://spamassassin.apache.org/old/publiccorpus/
Wykorzystać 2 dowolne klasy decyzyjne.

3. Zaimplementować funkcjonalność walidacji krzyżowej w celu dokładnego zweryfikowania parametrów klasyfikacji.
Walidacja krzyżowa polega na podziale systemu treningowego na k fragmentów, gdzie k-1 fragmentów służy jako nowy system
treningowy, a pozostała część służy jako system testowy przeznaczony do oceny parametrów klasyfikacji modelu.
Cała operacja jest powtarzana k-krotnie, gdzie każdy z pierwotnie wyznaczonych k fragmentów musi zostać użyty
jako system treningowy dokładnie jeden raz. Końcowy wynik walidacji krzyżowej to uśrednione parametry klasyfikacji
ze wszystkich k iteracji wraz z odchyleniami standardowymi. Przykład dla 100 obiektów treningowych, gdzie k=5. System treningowy zostaje podzielony na
5 fragmentów (1, 2, 3, 4, 5), gdzie każdy liczy 20 obiektów. Dokonujemy 5 iteracji:
    - iteracja 1: system treningowy tworzą fragmenty 2, 3, 4, 5; system testowy tworzy fragment 1; dokładność klasyfikacji wynosi 80%
    - iteracja 2: system treningowy tworzą fragmenty 1, 3, 4, 5; system testowy tworzy fragment 2; dokładność klasyfikacji wynosi 85%
    - iteracja 3: system treningowy tworzą fragmenty 1, 2, 4, 5; system testowy tworzy fragment 3; dokładność klasyfikacji wynosi 91%
    - iteracja 4: system treningowy tworzą fragmenty 1, 2, 3, 5; system testowy tworzy fragment 4; dokładność klasyfikacji wynosi 87%
    - iteracja 5: system treningowy tworzą fragmenty 1, 2, 3, 4; system testowy tworzy fragment 5; dokładność klasyfikacji wynosi 90%
Wynik końcowy to uśredniony wynik parametrów ze wszystkich iteracji, zatem dokładność końcowa wynosi 86.6%, przy odchyleniu standardowym 3.93


## Klasyfikator bayesowski w pakiecie scikit-learn

Scikit-learn to jedna z najpopularniejszych bibliotek zawierających zaimplementowane klasyfikatory i regresory oraz popularne systemy decyzyjne.

### Przykład naiwnej klasyfikacji bayesowskiej przy użyciu biblioteki scikit-learn:

#### Wczytanie systemu decyzyjnego

In [7]:
df = pd.read_csv('data/data-bayes.csv')

df.head()

,glucose,bloodpressure,diabetes
0,40,85,0
1,40,92,0
2,45,63,1
3,45,80,0
4,40,73,1


#### Przygotowanie systemu decyzyjnego do klasyfikacji

Podział na atrybuty wejściowe (X) i decyzyjny (y) oraz na system treningowy i testowy w stosunku 75%:25%

In [8]:
X = df.drop('diabetes', axis=1)
y = df['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

In [9]:
X_train.head(n=3)

,glucose,bloodpressure
639,45,83
37,40,65
346,50,85


In [10]:
y_test.head(n=3)

594    1
64     0
207    0
Name: diabetes, dtype: int64

#### Utworzenie modelu i uczenie

In [11]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

#### Walidacja dokładności wytrenowanego modelu

In [12]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100

print(f'Dokladnosc wytrenowanego modelu: {acc}%')


Dokladnosc wytrenowanego modelu: 94.77911646586345%


#### Predykcja klasy decyzyjnej obiektu

In [13]:
id_ = 10

pred = model.predict([X_test.iloc[id_].tolist()])
original_decision = y_test.iloc[id_]

print(f'Przydzielona decyzja: {pred[0]}, oryginalna decyzja: {original_decision}')

Przydzielona decyzja: 1, oryginalna decyzja: 1


### Zadania

1. Korzystając ze źródła http://archive.ics.uci.edu/ml/datasets.php wybrać jeden system decyzyjny, a następnie dokonać
klasyfikacji metodą Naiwnej Klasyfikacji Bayesowskiej dowolnego atrybutu decyzyjnego. Wykorzystując rozwiązania zadań z regresji i klasyfikacji KNN,
sprawdzić wyniki na następujących podziałach systemów decyzyjnych:
    - 55% system treningowy i 45% system testowy
    - 60% system treningowy i 40% system testowy
    - 65% system treningowy i 35% system testowy
    - 70% system treningowy i 30% system testowy
    - 75% system treningowy i 25% system testowy

2. Porównać wyniki uzyskane w zadaniu 1 z klasyfikatorem KNN
